In [1]:
# Import preliminaries
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import func
# from config import key

# EXTRACT 1: Create dataframes for health measurement data

In [2]:
# Import the annual_report csv file and inspect the columns
health_csv = "data/annual_report.csv"
health_df = pd.read_csv(health_csv)
health_df.head()

,Edition,Report Type,Measure Name,State Name,Rank,Value,Score,Lower CI,Upper CI,Source,Source Year
0,2019,2019 Annual,Adverse Childhood Experiences,Alaska,34.0,24.1,0.96,27.7,20.5,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
1,2019,2019 Annual,Adverse Childhood Experiences,Alabama,46.0,26.3,1.56,30.1,22.5,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
2,2019,2019 Annual,Adverse Childhood Experiences,United States,NaN,20.5,NaN,21.2,19.8,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
3,2019,2019 Annual,Adverse Childhood Experiences,Arkansas,47.0,27.1,1.76,30.8,23.3,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
4,2019,2019 Annual,Adverse Childhood Experiences,Arizona,48.0,27.3,1.82,31.1,23.4,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017


## TRANSFORM: Only keep columns of interest

In [3]:
# Only keep 6 columns: state name, rank, measure name, score, source, source year
health_df = health_df[['State Name', 'Rank', 'Measure Name', 'Score', 'Source', 'Source Year']].copy()
health_df.head()

,State Name,Rank,Measure Name,Score,Source,Source Year
0,Alaska,34.0,Adverse Childhood Experiences,0.96,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
1,Alabama,46.0,Adverse Childhood Experiences,1.56,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
2,United States,NaN,Adverse Childhood Experiences,NaN,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
3,Arkansas,47.0,Adverse Childhood Experiences,1.76,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
4,Arizona,48.0,Adverse Childhood Experiences,1.82,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017


## TRANSFORM: Isolate each Health Measure Name of interest 
* mental illness
* insufficient sleep
* air_pollution

In [4]:
# Create mental illness dataframe
health_mental_df = health_df.loc[health_df['Measure Name']=='Mental illness', :]
health_mental_df.head()

,State Name,Rank,Measure Name,Score,Source,Source Year
23598,Alaska,45.0,Mental illness,1.94,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
23599,Alabama,14.0,Mental illness,0.05,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
23600,United States,NaN,Mental illness,NaN,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
23601,Arkansas,41.0,Mental illness,1.59,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
23602,Arizona,28.0,Mental illness,0.45,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017


In [5]:
# Create insufficient sleep dataframe
health_sleep_df = health_df.loc[health_df['Measure Name']=='Insufficient Sleep', :]
health_sleep_df.head()

,State Name,Rank,Measure Name,Score,Source,Source Year
21310,Alabama,43.0,Insufficient Sleep,1.21,"CDC, Behavioral Risk Factor Surveillance System",2018
21311,Alaska,20.0,Insufficient Sleep,-0.30,"CDC, Behavioral Risk Factor Surveillance System",2018
21312,Arizona,22.0,Insufficient Sleep,-0.15,"CDC, Behavioral Risk Factor Surveillance System",2018
21313,Arkansas,29.0,Insufficient Sleep,0.47,"CDC, Behavioral Risk Factor Surveillance System",2018
21314,California,25.0,Insufficient Sleep,-0.03,"CDC, Behavioral Risk Factor Surveillance System",2018


In [6]:
# Create air pollution dataframe
health_airpollution_df = health_df.loc[health_df['Measure Name']=='Air Pollution', :]
health_airpollution_df.head()

,State Name,Rank,Measure Name,Score,Source,Source Year
52,Alaska,10.0,Air Pollution,-1.40,U.S. Environmental Protection Agency; U.S. Cen...,2016-2018
53,Alabama,36.0,Air Pollution,-0.21,U.S. Environmental Protection Agency; U.S. Cen...,2016-2018
54,Arkansas,19.0,Air Pollution,-0.91,U.S. Environmental Protection Agency; U.S. Cen...,2016-2018
55,Arizona,49.0,Air Pollution,0.91,U.S. Environmental Protection Agency; U.S. Cen...,2016-2018
56,California,50.0,Air Pollution,2.00,U.S. Environmental Protection Agency; U.S. Cen...,2016-2018


# TRANSFORM: Clean each dataframe. 
Define a function to perform the following: 
* Sort each dataframe by state
* Rename columns to match the SQL table column names
* Reset index for each dataframe.

In [7]:
# Function to clean dataframe
def clean_df(df, col_append):
    df = df.sort_values(by='State Name', ascending=False)
    df = df.rename(columns={"State Name":"state", 'Measure Name': 'measure_name', 'Source Year': 'source_year', "Rank":"rank", 'Source':'source','Score':"score"})
    df = df.reset_index(drop=True)
    df.columns = ['{}_'.format(col_append)+col_name for col_name in df.columns]
    return df

In [8]:
# Clean air pollution dataframe
health_airpollution_df = clean_df(health_airpollution_df, "pollution")
health_airpollution_df.head()
# health_airpollution_df.count()

,pollution_state,pollution_rank,pollution_measure_name,pollution_score,pollution_source,pollution_source_year
0,Wyoming,3.0,Air Pollution,-2.00,U.S. Environmental Protection Agency; U.S. Cen...,2016-2018
1,Wisconsin,15.0,Air Pollution,-1.12,U.S. Environmental Protection Agency; U.S. Cen...,2016-2018
2,West Virginia,29.0,Air Pollution,-0.56,U.S. Environmental Protection Agency; U.S. Cen...,2016-2018
3,Washington,34.0,Air Pollution,-0.28,U.S. Environmental Protection Agency; U.S. Cen...,2016-2018
4,Virginia,17.0,Air Pollution,-1.05,U.S. Environmental Protection Agency; U.S. Cen...,2016-2018


In [9]:
# Clean mental illness dataframe
health_mental_df = clean_df(health_mental_df, "mental")
health_mental_df.head()
# health_mental_df.count()

,mental_state,mental_rank,mental_measure_name,mental_score,mental_source,mental_source_year
0,Wyoming,42.0,Mental illness,1.74,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
1,Wisconsin,38.0,Mental illness,1.19,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
2,West Virginia,43.0,Mental illness,1.89,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
3,Washington,34.0,Mental illness,0.85,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
4,Virginia,13.0,Mental illness,-0.05,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017


In [10]:
# Clean insufficient sleep dataframe
health_sleep_df = clean_df(health_sleep_df, "sleep")
health_sleep_df.head()
# health_sleep_df.count()

,sleep_state,sleep_rank,sleep_measure_name,sleep_score,sleep_source,sleep_source_year
0,Wyoming,15.0,Insufficient Sleep,-0.47,"CDC, Behavioral Risk Factor Surveillance System",2018
1,Wisconsin,10.0,Insufficient Sleep,-0.74,"CDC, Behavioral Risk Factor Surveillance System",2018
2,West Virginia,49.0,Insufficient Sleep,2.00,"CDC, Behavioral Risk Factor Surveillance System",2018
3,Washington,7.0,Insufficient Sleep,-0.86,"CDC, Behavioral Risk Factor Surveillance System",2018
4,Virginia,41.0,Insufficient Sleep,1.03,"CDC, Behavioral Risk Factor Surveillance System",2018


# EXTRACT 2: Store State Happiness Ranking Data

In [11]:
happiness_csv = "data/happiness.csv"
happiness_df = pd.read_csv(happiness_csv)
happiness_df = happiness_df[['State', 'overall', 'emotAndPhysRank', 'Pop']].copy()
happiness_df = happiness_df.rename(columns={"State":"state", \
                                            "overall": "overall_rank",\
                                            "emotAndPhysRank": "emotphys_rank",\
                                           "Pop": "population"})
happiness_df = happiness_df.sort_values(by='state', ascending=True)
happiness_df = happiness_df.reset_index(drop=True)
happiness_df.head()

,state,overall_rank,emotphys_rank,population
0,Alabama,45,46,4898246
1,Alaska,47,33,735720
2,Arizona,21,27,7275070
3,Arkansas,49,50,3026412
4,California,5,4,39747267


# EXTRACT 3: State Area Data 

In [12]:
area = "data/state_area.csv"
area_df = pd.read_csv(area)

## TRANSFORM: Only keep columns of interest and join population data with the state area dataframe

In [13]:
# Keep state, total area, latitude and longitude and rename columns
state_area = area_df[["State", "Internal_lat", "Internal_long", "Total_km2"]].copy()
state_area = state_area.rename(columns={"State": "state", "Total_km2": "area_km2"})

# Join the population column from the happiness dataframe and calculate area per capita
population = happiness_df[["state", "population"]]
state_census = state_area.merge(population, on="state")
state_census["area_km2"] = state_census["area_km2"].str.replace(",","").astype(float)
state_census["area_per_capita"] = state_census["area_km2"] / state_census["population"]

# See the first five rows of the updated dataframe
state_census.head()

,state,Internal_lat,Internal_long,area_km2,population,area_per_capita
0,Alabama,32.739632,-86.843459,135767.0,4898246,0.027717
1,Alaska,63.346191,-152.837068,1723337.0,735720,2.342382
2,Arizona,34.209964,-111.602401,295234.0,7275070,0.040582
3,Arkansas,34.895526,-92.444626,137732.0,3026412,0.045510
4,California,37.148573,-119.540651,423967.0,39747267,0.010667


# LOAD: Load dataframes to SQL database

In [14]:
rds_connection_string = "postgres:{key}@localhost:5432/happy_US"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [15]:
engine.table_names()

['mental_illness', 'insufficient_sleep', 'air_pollution', 'happiness_score']

In [16]:
health_mental_df.to_sql(name='mental_illness', con=engine, if_exists='append', index=False)  
health_sleep_df.to_sql(name='insufficient_sleep', con=engine, if_exists='append', index=False) 
health_airpollution_df.to_sql(name='air_pollution', con=engine, if_exists='append', index=False) 
happiness_df.to_sql(name='happiness_score', con=engine, if_exists='append', index=False) 

In [18]:
pd.read_sql_query('select * from mental_illness', con=engine).head()

,mental_state,mental_measure_name,mental_rank,mental_score,mental_source,mental_source_year
0,Wyoming,Mental illness,42.0,1.74,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
1,Wisconsin,Mental illness,38.0,1.19,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
2,West Virginia,Mental illness,43.0,1.89,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
3,Washington,Mental illness,34.0,0.85,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017
4,Virginia,Mental illness,13.0,-0.05,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017


In [19]:
pd.read_sql_query('select * from insufficient_sleep', con=engine).head()

,sleep_state,sleep_measure_name,sleep_rank,sleep_score,sleep_source,sleep_source_year
0,Wyoming,Insufficient Sleep,15.0,-0.47,"CDC, Behavioral Risk Factor Surveillance System",2018
1,Wisconsin,Insufficient Sleep,10.0,-0.74,"CDC, Behavioral Risk Factor Surveillance System",2018
2,West Virginia,Insufficient Sleep,49.0,2.00,"CDC, Behavioral Risk Factor Surveillance System",2018
3,Washington,Insufficient Sleep,7.0,-0.86,"CDC, Behavioral Risk Factor Surveillance System",2018
4,Virginia,Insufficient Sleep,41.0,1.03,"CDC, Behavioral Risk Factor Surveillance System",2018


In [20]:
pd.read_sql_query('select * from air_pollution', con=engine).head()

,pollution_state,pollution_measure_name,pollution_rank,pollution_score,pollution_source,pollution_source_year
0,Wyoming,Air Pollution,3.0,-2.00,U.S. Environmental Protection Agency; U.S. Cen...,2016-2018
1,Wisconsin,Air Pollution,15.0,-1.12,U.S. Environmental Protection Agency; U.S. Cen...,2016-2018
2,West Virginia,Air Pollution,29.0,-0.56,U.S. Environmental Protection Agency; U.S. Cen...,2016-2018
3,Washington,Air Pollution,34.0,-0.28,U.S. Environmental Protection Agency; U.S. Cen...,2016-2018
4,Virginia,Air Pollution,17.0,-1.05,U.S. Environmental Protection Agency; U.S. Cen...,2016-2018


In [21]:
pd.read_sql_query('select * from happiness_score', con=engine).head()

,overall_rank,emotphys_rank,state,population
0,45,46,Alabama,4898246
1,47,33,Alaska,735720
2,21,27,Arizona,7275070
3,49,50,Arkansas,3026412
4,5,4,California,39747267


In [30]:
pd.read_sql_query('select distinct h.state, * from happiness_score h inner join air_pollution p on h.state = p.pollution_state \
inner join mental_illness m on h.state =m.mental_state inner join insufficient_sleep s on h.state=s.sleep_state \
order by h.state asc'
                  , con=engine).head()

,state,overall_rank,emotphys_rank,state,population,pollution_state,pollution_measure_name,pollution_rank,pollution_score,pollution_source,...,mental_rank,mental_score,mental_source,mental_source_year,sleep_state,sleep_measure_name,sleep_rank,sleep_score,sleep_source,sleep_source_year
0,Alabama,45,46,Alabama,4898246,Alabama,Air Pollution,36,-0.21,U.S. Environmental Protection Agency; U.S. Cen...,...,14,0.05,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017,Alabama,Insufficient Sleep,43,1.21,"CDC, Behavioral Risk Factor Surveillance System",2018
1,Alaska,47,33,Alaska,735720,Alaska,Air Pollution,10,-1.40,U.S. Environmental Protection Agency; U.S. Cen...,...,45,1.94,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017,Alaska,Insufficient Sleep,20,-0.30,"CDC, Behavioral Risk Factor Surveillance System",2018
2,Arizona,21,27,Arizona,7275070,Arizona,Air Pollution,49,0.91,U.S. Environmental Protection Agency; U.S. Cen...,...,28,0.45,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017,Arizona,Insufficient Sleep,22,-0.15,"CDC, Behavioral Risk Factor Surveillance System",2018
3,Arkansas,49,50,Arkansas,3026412,Arkansas,Air Pollution,19,-0.91,U.S. Environmental Protection Agency; U.S. Cen...,...,41,1.59,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017,Arkansas,Insufficient Sleep,29,0.47,"CDC, Behavioral Risk Factor Surveillance System",2018
4,California,5,4,California,39747267,California,Air Pollution,50,2.00,U.S. Environmental Protection Agency; U.S. Cen...,...,3,-1.00,"U.S. HHS, HRSA, Maternal and Child Health Bure...",2016-2017,California,Insufficient Sleep,25,-0.03,"CDC, Behavioral Risk Factor Surveillance System",2018
